In [1]:
# 체 Field
'''
사칙 연산이 정의되는 집합
원소의 개수가 유한개인 체 := 유한체
'''

In [4]:
# 유한체 (Finite Field)
p = 7
F_p = [i for i in range(p)]

In [67]:
# 모듈러 연산(== 나머지 연산) 
def Modular(a:int, b:int, cal_op:['+', '-','/', '*'], mod:int):
    if cal_op == '+':
        result = (a + b) % mod
    elif cal_op == '-':
        result = (a - b) % mod
    elif cal_op == '*':
        result = (a * b) % mod
    elif cal_op == '/':
        for i in range(mod):
            if (b * i) % mod == 1:
                b = i
                break        
        result = (a * b) % mod
    else:
        raise Exception("연산기호가 맞지 않습니다. : '+', '-','/', '*' ")
        
    return result

In [19]:
Modular(3, 5, '/', 7)

2

In [ ]:
##################################### Elliptic curve Modular Arithmethic ################################

In [27]:
from typing import *

In [120]:
def EllipticModular(P1:List, P2:List, cal_op:['neg', 'add','mul'], mod:int, a, b):
    """
    P1, P2 := (x, y)
    cal_op := ['neg', 'add','mul']
        neg == negation 
            neg P = -P
        add == addition 
            P1 + P2 = R
        mul == scalar multiplication
            scalr * P1
    mod := int
    y ** 2 = x ** 3 + a*x + b
    4 * (a**3) + 27 * (b ** 2) != 0
    """
    
    assert (isinstance(P1, List)) \
            and (isinstance(P2, List) or P2 is None), "P1 must be List and P2 must be List or None" 
    
    assert 4 * (a**3) + 27 * (b ** 2) != 0, 'Avoid Crossing in Elliptic!'
    
    if cal_op == 'add':
        s = Modular((P2[1] - P1[1]), (P2[0] - P1[0]), '/', mod)
        xr = Modular(s ** 2 - P1[0], P2[0], '-', mod)
        yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
        result = (xr, yr)

    elif cal_op == 'mul':
        scalar = int(input("Scalar : "))
        assert isinstance(scalar, int), "scalar must be positive int !"
        if scalar == 2:
            s = Modular((3*(P1[0] ** 2) + a), (2 * P1[1]), '/', mod)
            xr = Modular(s ** 2, 2*P1[0], '-', mod)
            yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
            result = (xr, yr)
        else:
            s = Modular((3*(P1[0] ** 2) + a), (2 * P1[1]), '/', mod)
            xr = Modular(s ** 2, 2*P1[0], '-', mod)
            yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
            P_ = [xr, yr]
            for i in range(1, scalar - 1):
                s = Modular(P1[1] - P_[1], P1[0] - P_[0], '/', mod)
                xr = Modular(s ** 2 - P1[0], P_[0], '-', mod)
                yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
                P_ = [xr, yr]
            result = (P_[0], P_[1])
    else:
        raise Exception("연산기호가 맞지 않습니다. : ['neg', 'add','mul'] ")
        
    return result

In [121]:
EllipticModular(P1=[15, 13], P2=None, cal_op='mul', mod=17, a=0, b=7)

Scalar : 3


(8, 3)

In [118]:
EllipticModular(P1=[15, 13], P2=[12, 1], cal_op='add', mod=17, a=0, b=7)

(6, 6)